<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_10_28_Extract_Markdown_from_PDF_with_Vision_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to ingest a PDF from a URL, and extract as Markdown format using the latest vision models, such as Anthropic Sonnet 3.5.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [9]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [10]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [11]:
async def create_openai_specification(model: enums.OpenAIModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"OpenAI [{model}]",
        type=enums.SpecificationTypes.PREPARATION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_anthropic_specification(model: enums.AnthropicModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Anthropic [{model}]",
        type=enums.SpecificationTypes.PREPARATION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_google_specification(model: enums.GoogleModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Google [{model}]",
        type=enums.SpecificationTypes.PREPARATION,
        serviceType=enums.ModelServiceTypes.GOOGLE,
        google=input_types.GoogleModelPropertiesInput(
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_workflow(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Document Preparation",
        preparation=input_types.PreparationWorkflowStageInput(
            jobs=[
                input_types.PreparationWorkflowJobInput(
                    connector=input_types.FilePreparationConnectorInput(
                        type=enums.FilePreparationServiceTypes.MODEL_DOCUMENT,
                        modelDocument=input_types.ModelDocumentPreparationPropertiesInput(
                            specification=input_types.EntityReferenceInput(id=specification_id)
                        )
                    )
                )
            ]
        )
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def ingest_uri(uri: str, workflow_id: str):
    if graphlit.client is None:
        return

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, workflow=input_types.EntityReferenceInput(id=workflow_id), is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def get_content(content_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.get_content(content_id)

        return response.content
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_content(content_id: str):
    if graphlit.client is None:
        return;

    try:
        _ = await graphlit.client.delete_content(content_id)

        return
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return

# NOTE: these functions are just used to clean-up old data before executing the example
async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)


Execute Graphlit example

In [12]:
from IPython.display import display, Markdown

uri="https://graphlitplatform.blob.core.windows.net/test/documents/preparation-sample.pdf"


Extract PDF to Markdown using Anthropic Sonnet 3.5 (2024-06-20 version)

In [13]:
# Remove any existing contents, specifications and workflows; only needed for notebook example
await delete_all_contents()
await delete_all_specifications()
await delete_all_workflows()

print('Deleted all contents, specifications and workflows.')

specification_id = await create_anthropic_specification(enums.AnthropicModels.CLAUDE_3_5_SONNET_20240620)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        content_id = await ingest_uri(uri, workflow_id)

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            content = await get_content(content_id)

            if content is not None:
                display(Markdown(content.markdown))

                print('-----------------------------------------------------------------------------')
                print(content.markdown)
                print('-----------------------------------------------------------------------------')

            await delete_content(content_id)


Deleted all contents, specifications and workflows.
Created specification [7ee99c28-f298-4d38-9ff8-e755e9e8a43a].
Created workflow [880066ab-c229-4b20-9566-57e9f08817c6].
Ingested content [f288189d-09ff-4827-b9cd-018bdcb30340].


# Item 1. Financial Statements

## Consolidated Balance Sheets

(dollars in thousands)
(unaudited)

|  | June 30, 2024 | December 31, 2023 
| --- | --- | --- |
| ASSETS |  |  |
| Current Assets |  |  |
| Cash and Cash Equivalents | $ 307,433 | $ 383,180 |
| Investments—Consolidated Investment Companies | 72,847 | 70,543 |
| Investments—Affiliates and Other | 72,599 | 106,952 |
| Receivables, net of reserve of $21 and $21, respectively | 72,086 | 75,721 |
| Receivables—Affiliates | 44,472 | 48,694 |
| Prepaid Expenses | 37,911 | 29,739 |
| Other Current Assets | 5,953 | 5,900 |
| Total Current Assets | 613,301 | 720,729 |
| Long-Term Assets |  |  |
| Goodwill | 806,069 | 807,156 |
| Intangible Assets, net of accumulated amortization of $70,045 and $64,112, respectively | 335,554 | 409,449 |
| Property and Equipment, net of accumulated depreciation of $122,372 and $119,852, respectively | 32,170 | 30,711 |
| Right-of-Use Assets, net | 92,464 | 99,265 |
| Other Long-Term Assets | 31,949 | 34,534 |
| Total Long-Term Assets | 1,298,206 | 1,381,115 |
| Total Assets | $ 1,911,507 | $ 2,101,844 |
| LIABILITIES |  |  |
| Current Liabilities |  |  |
| Accounts Payable and Accrued Expenses | $ 92,564 | $ 88,290 |
| Accrued Compensation and Benefits | 93,153 | 158,392 |
| Lease Liabilities | 16,437 | 16,283 |
| Other Current Liabilities | 21,226 | 24,378 |
| Total Current Liabilities | 223,380 | 287,343 |
| Long-Term Liabilities |  |  |
| Long-Term Debt | 347,974 | 347,843 |
| Long-Term Deferred Tax Liability, net | 181,403 | 186,292 |
| Long-Term Lease Liabilities | 85,748 | 93,816 |
| Other Long-Term Liabilities | 25,486 | 32,453 |
| Total Long-Term Liabilities | 640,611 | 660,404 |
| Total Liabilities | 863,991 | 947,747 |
| Commitments and Contingencies (Note (17)) |  |  |
| TEMPORARY EQUITY |  |  |
| Redeemable Noncontrolling Interests in Subsidiaries | 30,491 | 25,845 |
| PERMANENT EQUITY |  |  |
| Federated Hermes, Inc. Shareholders' Equity |  |  |
| Common Stock: |  |  |
| Class A, No Par Value, 20,000 Shares Authorized, 9,000 Shares Issued and Outstanding | 189 | 189 |
| Class B, No Par Value, 900,000,000 Shares Authorized, 99,505,456 Shares Issued | 491,204 | 474,814 |
| Additional Paid-In Capital from Treasury Stock Transactions | 0 | 2 |
| Retained Earnings | 1,145,844 | 1,194,561 |
| Treasury Stock, at Cost, 16,905,080 and 14,664,467 Shares Class B Common Stock, respectively | (595,592) | (521,403) |
| Accumulated Other Comprehensive Income (Loss), net of tax | (24,620) | (19,911) |
| Total Permanent Equity | 1,017,025 | 1,128,252 |
| Total Liabilities, Temporary Equity and Permanent Equity | $ 1,911,507 | $ 2,101,844 |


(The accompanying notes are an integral part of these Consolidated Financial Statements.)

4



-----------------------------------------------------------------------------
# Item 1. Financial Statements

## Consolidated Balance Sheets

(dollars in thousands)
(unaudited)

|  | June 30, 2024 | December 31, 2023 
| --- | --- | --- |
| ASSETS |  |  |
| Current Assets |  |  |
| Cash and Cash Equivalents | $ 307,433 | $ 383,180 |
| Investments—Consolidated Investment Companies | 72,847 | 70,543 |
| Investments—Affiliates and Other | 72,599 | 106,952 |
| Receivables, net of reserve of $21 and $21, respectively | 72,086 | 75,721 |
| Receivables—Affiliates | 44,472 | 48,694 |
| Prepaid Expenses | 37,911 | 29,739 |
| Other Current Assets | 5,953 | 5,900 |
| Total Current Assets | 613,301 | 720,729 |
| Long-Term Assets |  |  |
| Goodwill | 806,069 | 807,156 |
| Intangible Assets, net of accumulated amortization of $70,045 and $64,112, respectively | 335,554 | 409,449 |
| Property and Equipment, net of accumulated depreciation of $122,372 and $119,852, respectively | 32,170 | 30,711 |
| Ri

Extract PDF to Markdown using Anthropic Sonnet 3.5 (2024-10-22 version)

In [14]:
# Remove any existing contents, specifications and workflows; only needed for notebook example
await delete_all_contents()
await delete_all_specifications()
await delete_all_workflows()

print('Deleted all contents, specifications and workflows.')

specification_id = await create_anthropic_specification(enums.AnthropicModels.CLAUDE_3_5_SONNET_20241022)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        content_id = await ingest_uri(uri, workflow_id)

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            content = await get_content(content_id)

            if content is not None:
                display(Markdown(content.markdown))

                print('-----------------------------------------------------------------------------')
                print(content.markdown)
                print('-----------------------------------------------------------------------------')

            await delete_content(content_id)


Deleted all contents, specifications and workflows.
Created specification [5a0ab9cc-a41f-490b-966b-a5ebc7ad3336].
Created workflow [929f6cb6-87e2-4098-83b6-d361bb33549e].
Ingested content [f22f8342-7fe3-459f-a7f3-9674356411f5].


# Item 1. Financial Statements

## Consolidated Balance Sheets

(dollars in thousands)
(unaudited)

|  | June 30, 2024 | December 31, 2023 
| --- | --- | --- |
| ASSETS |  |  |
| Current Assets |  |  |
| Cash and Cash Equivalents | $ 307,433 | $ 383,180 |
| Investments—Consolidated Investment Companies | 72,847 | 70,543 |
| Investments—Affiliates and Other | 72,599 | 106,952 |
| Receivables, net of reserve of $21 and $21, respectively | 72,086 | 75,721 |
| Receivables—Affiliates | 44,472 | 48,694 |
| Prepaid Expenses | 37,911 | 29,739 |
| Other Current Assets | 5,953 | 5,900 |
| Total Current Assets | 613,301 | 720,729 |
| Long-Term Assets |  |  |
| Goodwill | 806,069 | 807,156 |
| Intangible Assets, net of accumulated amortization of $70,045 and $64,112, respectively | 335,554 | 409,449 |
| Property and Equipment, net of accumulated depreciation of $122,372 and $119,852, respectively | 32,170 | 30,711 |
| Right-of-Use Assets, net | 92,464 | 99,265 |
| Other Long-Term Assets | 31,949 | 34,534 |
| Total Long-Term Assets | 1,298,206 | 1,381,115 |
| Total Assets | $ 1,911,507 | $ 2,101,844 |
| LIABILITIES |  |  |
| Current Liabilities |  |  |
| Accounts Payable and Accrued Expenses | $ 92,564 | $ 88,290 |
| Accrued Compensation and Benefits | 93,153 | 158,392 |
| Lease Liabilities | 16,437 | 16,283 |
| Other Current Liabilities | 21,226 | 24,378 |
| Total Current Liabilities | 223,380 | 287,343 |
| Long-Term Liabilities |  |  |
| Long-Term Debt | 347,974 | 347,843 |
| Long-Term Deferred Tax Liability, net | 181,403 | 186,292 |
| Long-Term Lease Liabilities | 85,748 | 93,816 |
| Other Long-Term Liabilities | 25,486 | 32,453 |
| Total Long-Term Liabilities | 640,611 | 660,404 |
| Total Liabilities | 863,991 | 947,747 |
| Commitments and Contingencies (Note (17)) |  |  |
| TEMPORARY EQUITY |  |  |
| Redeemable Noncontrolling Interests in Subsidiaries | 30,491 | 25,845 |
| PERMANENT EQUITY |  |  |
| Federated Hermes, Inc. Shareholders' Equity |  |  |
| Common Stock: |  |  |
| Class A, No Par Value, 20,000 Shares Authorized, 9,000 Shares Issued and Outstanding | 189 | 189 |
| Class B, No Par Value, 900,000,000 Shares Authorized, 99,505,456 Shares Issued | 491,204 | 474,814 |
| Additional Paid-In Capital from Treasury Stock Transactions | 0 | 2 |
| Retained Earnings | 1,145,844 | 1,194,561 |
| Treasury Stock, at Cost, 16,905,080 and 14,664,467 Shares Class B Common Stock, respectively | (595,592) | (521,403) |
| Accumulated Other Comprehensive Income (Loss), net of tax | (24,620) | (19,911) |
| Total Permanent Equity | 1,017,025 | 1,128,252 |
| Total Liabilities, Temporary Equity and Permanent Equity | $ 1,911,507 | $ 2,101,844 |


(The accompanying notes are an integral part of these Consolidated Financial Statements.)



-----------------------------------------------------------------------------
# Item 1. Financial Statements

## Consolidated Balance Sheets

(dollars in thousands)
(unaudited)

|  | June 30, 2024 | December 31, 2023 
| --- | --- | --- |
| ASSETS |  |  |
| Current Assets |  |  |
| Cash and Cash Equivalents | $ 307,433 | $ 383,180 |
| Investments—Consolidated Investment Companies | 72,847 | 70,543 |
| Investments—Affiliates and Other | 72,599 | 106,952 |
| Receivables, net of reserve of $21 and $21, respectively | 72,086 | 75,721 |
| Receivables—Affiliates | 44,472 | 48,694 |
| Prepaid Expenses | 37,911 | 29,739 |
| Other Current Assets | 5,953 | 5,900 |
| Total Current Assets | 613,301 | 720,729 |
| Long-Term Assets |  |  |
| Goodwill | 806,069 | 807,156 |
| Intangible Assets, net of accumulated amortization of $70,045 and $64,112, respectively | 335,554 | 409,449 |
| Property and Equipment, net of accumulated depreciation of $122,372 and $119,852, respectively | 32,170 | 30,711 |
| Ri

Extract PDF to Markdown using Google Gemini 1.5 Pro

In [15]:
specification_id = await create_google_specification(enums.GoogleModels.GEMINI_1_5_PRO_002)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        content_id = await ingest_uri(uri, workflow_id)

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            content = await get_content(content_id)

            if content is not None:
                display(Markdown(content.markdown))

                print('-----------------------------------------------------------------------------')
                print(content.markdown)
                print('-----------------------------------------------------------------------------')

            await delete_content(content_id)


Created specification [5e253602-3579-4b3a-84f6-3095e8129fa2].
Created workflow [81de74b1-87a9-48cf-bc3d-7309ff93e410].
Ingested content [0dc28ec2-c88d-4dba-805e-c1de754388c5].


## Item 1. Financial Statements

### Consolidated Balance Sheets

(dollars in thousands)
(unaudited)

 | June 30,
2024 | December 31,
2023 
| --- | --- |
| ASSETS |  |  |
| Current Assets |  |  |
| Cash and Cash Equivalents | $ 307,433 | $ 383,180 |
| Investments—Consolidated Investment Companies | 72,847 | 70,543 |
| Investments—Affiliates and Other | 72,599 | 106,952 |
| Receivables, net of reserve of $21 and $21, respectively | 72,086 | 75,721 |
| Receivables—Affiliates | 44,472 | 48,694 |
| Prepaid Expenses | 37,911 | 29,739 |
| Other Current Assets | 5,953 | 5,900 |
| Total Current Assets | 613,301 | 720,729 |
| Long-Term Assets |  |  |
| Goodwill | 806,069 | 807,156 |
| Intangible Assets, net of accumulated amortization of $70,045 and $64,112, respectively | 335,554 | 409,449 |
| Property and Equipment, net of accumulated depreciation of $122,372 and $119,852, respectively | 32,170 | 30,711 |
| Right-of-Use Assets, net | 92,464 | 99,265 |
| Other Long-Term Assets | 31,949 | 34,534 |
| Total Long-Term Assets | 1,298,206 | 1,381,115 |
| Total Assets | $ 1,911,507 | $ 2,101,844 |


(The accompanying notes are an integral part of these Consolidated Financial Statements.)



-----------------------------------------------------------------------------
## Item 1. Financial Statements

### Consolidated Balance Sheets

(dollars in thousands)
(unaudited)

 | June 30,
2024 | December 31,
2023 
| --- | --- |
| ASSETS |  |  |
| Current Assets |  |  |
| Cash and Cash Equivalents | $ 307,433 | $ 383,180 |
| Investments—Consolidated Investment Companies | 72,847 | 70,543 |
| Investments—Affiliates and Other | 72,599 | 106,952 |
| Receivables, net of reserve of $21 and $21, respectively | 72,086 | 75,721 |
| Receivables—Affiliates | 44,472 | 48,694 |
| Prepaid Expenses | 37,911 | 29,739 |
| Other Current Assets | 5,953 | 5,900 |
| Total Current Assets | 613,301 | 720,729 |
| Long-Term Assets |  |  |
| Goodwill | 806,069 | 807,156 |
| Intangible Assets, net of accumulated amortization of $70,045 and $64,112, respectively | 335,554 | 409,449 |
| Property and Equipment, net of accumulated depreciation of $122,372 and $119,852, respectively | 32,170 | 30,711 |
| Right-of

Extract PDF to Markdown using OpenAI GPT-4o

In [8]:
# Remove any existing contents, specifications and workflows; only needed for notebook example
await delete_all_contents()
await delete_all_specifications()
await delete_all_workflows()

print('Deleted all contents, specifications and workflows.')

specification_id = await create_openai_specification(enums.OpenAIModels.GPT4O_128K)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        content_id = await ingest_uri(uri, workflow_id)

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            content = await get_content(content_id)

            if content is not None:
                display(Markdown(content.markdown))

                print('-----------------------------------------------------------------------------')
                print(content.markdown)
                print('-----------------------------------------------------------------------------')

            await delete_content(content_id)


Created workflow [fea13602-5d4b-4bcf-a3f7-4a8320c9ceb4].
Ingested content [c72e9d51-44c5-4520-a49a-d531b30c9de4].


## Item 1. Financial Statements

# Consolidated Balance Sheets

(dollars in thousands)

(unaudited)

## ASSETS

### Current Assets

Cash and Cash Equivalents

Investments—Consolidated Investment Companies

Investments—Affiliates and Other

Receivables, net of reserve of $21 and $21, respectively

Receivables—Affiliates

Prepaid Expenses

Other Current Assets

Total Current Assets

### Long-Term Assets

Goodwill

Intangible Assets, net of accumulated amortization of $70,045 and $64,112, respectively

Property and Equipment, net of accumulated depreciation of $122,372 and $119,852, respectively

Right-of-Use Assets, net

Other Long-Term Assets

Total Long-Term Assets

Total Assets

## LIABILITIES

### Current Liabilities

Accounts Payable and Accrued Expenses

Accrued Compensation and Benefits

Lease Liabilities

Other Current Liabilities

Total Current Liabilities

### Long-Term Liabilities

Long-Term Debt

Long-Term Deferred Tax Liability, net

Long-Term Lease Liabilities

Other Long-Term Liabilities

Total Long-Term Liabilities

Total Liabilities

Commitments and Contingencies (Note (17))

## TEMPORARY EQUITY

Redeemable Noncontrolling Interests in Subsidiaries

## PERMANENT EQUITY

Federated Hermes, Inc. Shareholders’ Equity

Common Stock:

Class A, No Par Value, 20,000 Shares Authorized, 9,000 Shares Issued and Outstanding

Class B, No Par Value, 900,000,000 Shares Authorized, 99,505,456 Shares Issued

Additional Paid-In Capital from Treasury Stock Transactions

Retained Earnings

Treasury Stock, at Cost, 16,905,080 and 14,664,467 Shares Class B Common Stock, respectively

Accumulated Other Comprehensive Income (Loss), net of tax

Total Permanent Equity

Total Liabilities, Temporary Equity and Permanent Equity

(The accompanying notes are an integral part of these Consolidated Financial Statements.)

| June 30, 2024 | December 31, 2023 
| --- | --- |
| $ 307,433 | $ 383,180 |
| 72,847 | 70,543 |
| 72,599 | 106,952 |
| 72,086 | 75,721 |
| 44,472 | 48,694 |
| 37,911 | 29,739 |
| 5,953 | 5,900 |
| 613,301 | 720,729 |
| 806,069 | 807,156 |
| 335,554 | 409,449 |
| 32,170 | 30,711 |
| 92,464 | 99,265 |
| 31,949 | 34,534 |
| 1,298,206 | 1,381,115 |
| 1,911,507 | 2,101,844 |
| 92,564 | 88,290 |
| 93,153 | 158,392 |
| 16,437 | 16,283 |
| 21,226 | 24,378 |
| 223,380 | 287,343 |
| 347,974 | 347,843 |
| 181,403 | 186,292 |
| 85,748 | 93,816 |
| 25,486 | 32,453 |
| 640,611 | 660,404 |
| 863,991 | 947,747 |
| 30,491 | 25,845 |
| 189 | 189 |
| 491,204 | 474,814 |
| 0 | 2 |
| 1,145,844 | 1,194,561 |
| (595,592) | (521,403) |
| (24,620) | (19,911) |
| 1,017,025 | 1,128,252 |
| 1,911,507 | 2,101,844 |




## Item 1. Financial Statements

# Consolidated Balance Sheets

(dollars in thousands)

(unaudited)

## ASSETS

### Current Assets

Cash and Cash Equivalents

Investments—Consolidated Investment Companies

Investments—Affiliates and Other

Receivables, net of reserve of $21 and $21, respectively

Receivables—Affiliates

Prepaid Expenses

Other Current Assets

Total Current Assets

### Long-Term Assets

Goodwill

Intangible Assets, net of accumulated amortization of $70,045 and $64,112, respectively

Property and Equipment, net of accumulated depreciation of $122,372 and $119,852, respectively

Right-of-Use Assets, net

Other Long-Term Assets

Total Long-Term Assets

Total Assets

## LIABILITIES

### Current Liabilities

Accounts Payable and Accrued Expenses

Accrued Compensation and Benefits

Lease Liabilities

Other Current Liabilities

Total Current Liabilities

### Long-Term Liabilities

Long-Term Debt

Long-Term Deferred Tax Liability, net

Long-Term Lease Liabilities

Other Long-T